In [ ]:
import numpy as np
import h5py
import h5flow
import matplotlib.pyplot as plt
import sys

In [ ]:
x_boundaries = np.array([-30.431, 30.431])
y_boundaries = np.array([-61.8543, 61.8543])
z_boundaries = np.array([-30.8163, 30.8163])

In [ ]:
common_path = '/global/cfs/cdirs/dune/www/data/ModuleX/'
runlist_path = 'runlist/'
runlist_name = 'runlist_flow_0-5_Efield_1.txt'
output_path = 'plots/occupancy/'
output_prefix = 'occupancy_'

In [ ]:
f_runlist = open(f'{common_path}{runlist_path}{runlist_name}', 'r')
names = f_runlist.read().splitlines()

In [ ]:
for i_f, f_name in enumerate(names):
    if i_f > 0:
        break
    print(f_name)
    f_label = f_name[-24:-3]
    out = f"{output_path}{output_prefix}{f_label}"
    f_manager = h5flow.data.H5FlowDataManager(f_name, 'r')

    PromptHits = f_manager["charge/events", "charge/calib_prompt_hits", 0:5000]

    tpc1_mask = (PromptHits['x']<0)
    # arr_z_tpc1 = PromptHits['z'][tpc1_mask]
    # arr_y_tpc1 = PromptHits['y'][tpc1_mask]

    tpc2_mask = (PromptHits['x']>0)
    # arr_z_tpc2 = PromptHits['z'][tpc2_mask]
    # arr_y_tpc2 = PromptHits['y'][tpc2_mask]    
    
    ########
    # TPC1
    ########
    h_tpc1_count, zedges, yedges = np.histogram2d(PromptHits['z'][tpc1_mask], PromptHits['y'][tpc1_mask], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
    Z, Y = np.meshgrid(zedges, yedges)
    plt.figure(figsize=(6,10))
    # c_tpc1_count = plt.pcolormesh(Z, Y, h_tpc1_count.T)
    c_tpc1_count = plt.pcolormesh(Z, Y, h_tpc1_count.T, vmax=800)
    plt.colorbar(c_tpc1_count)
    plt.xlabel("z [cm]")
    plt.ylabel("y [cm]")
    plt.title("Occupancy in TPC 1")
    plt.margins(0.2)
    plt.set_cmap("Greens")
    plt.savefig(f"{output_path}TPC1/{output_prefix}TPC1_{f_label}.pdf")
    # plt.show()
    
    ########
    # TPC2
    ########
    h_tpc2_count, zedges, yedges = np.histogram2d(PromptHits['z'][tpc2_mask], PromptHits['y'][tpc2_mask], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
    Z, Y = np.meshgrid(zedges, yedges)
    plt.figure(figsize=(6,10))
    c_tpc2_count = plt.pcolormesh(Z, Y, h_tpc2_count.T, vmax=800)
    plt.colorbar(c_tpc2_count)
    plt.xlabel("z [cm]")
    plt.ylabel("y [cm]")
    plt.title("Occupancy in TPC 2")
    plt.margins(0.2)
    plt.set_cmap("Greens")
    plt.savefig(f"{output_path}TPC2/{output_prefix}TPC2_{f_label}.pdf")
    # plt.show()
    
    # ########
    # # z
    # ########
    h_z_count, zedges, yedges = np.histogram2d(PromptHits['x'].flatten(), PromptHits['y'].flatten(), bins=[140,280], range=[[x_boundaries[0],x_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
    Z, Y = np.meshgrid(zedges, yedges)
    plt.figure(figsize=(6,10))
    c_z_count = plt.pcolormesh(Z, Y, h_z_count.T, vmax=800)
    plt.colorbar(c_z_count)
    plt.xlabel("x [cm]")
    plt.ylabel("y [cm]")
    plt.title("Occupancy (z projection)")
    plt.margins(0.2)
    plt.set_cmap("Greens")
    plt.savefig(f"{output_path}z_proj/{output_prefix}z_proj_{f_label}.pdf")
    # plt.show()
    
    ########
    # y
    ########
    h_y_count, zedges, yedges = np.histogram2d(PromptHits['x'].flatten(), PromptHits['z'].flatten(), bins=[140,140], range=[[z_boundaries[0],z_boundaries[1]],[z_boundaries[0],z_boundaries[1]]])
    Z, Y = np.meshgrid(zedges, yedges)
    plt.figure(figsize=(5,4))
    c_y_count = plt.pcolormesh(Z, Y, h_y_count.T, vmax=800)
    plt.colorbar(c_y_count)
    plt.xlabel("x [cm]")
    plt.ylabel("z [cm]")
    plt.title("Occupancy (y projection)")
    plt.margins(0.2)
    plt.set_cmap("Greens")
    plt.savefig(f"{output_path}y_proj/{output_prefix}y_proj_{f_label}.pdf")
    # plt.show()

In [ ]:
evts_per_load = 5000
for i_f, f_name in enumerate(names):
    print(f_name)
    f_label = f_name[-24:-3]
    out = f"{output_path}{output_prefix}{f_label}"
    f_manager = h5flow.data.H5FlowDataManager(f_name, 'r')

    evts = f_manager["charge/events/data"]

    total_evts = np.max(evts['id'])
    n_loads = np.ceil(total_evts/evts_per_load)
    total_h_tpc1_count = np.array([])
    total_h_tpc2_count = np.array([])
    total_h_z_count = np.array([])
    total_h_y_count = np.array([])
    
    for i_load in range(int(n_loads)):
        
        print("----i_load: ", i_load)
        
        start = int(evts_per_load*i_load)
        if i_load == n_loads - 1:
            end = int(total_evts)
        else:
            end = int(evts_per_load*(i_load+1))

        PromptHits = f_manager["charge/events", "charge/calib_prompt_hits", start:end]

        tpc1_mask = (PromptHits['x']<0)
        # arr_z_tpc1 = PromptHits['z'][tpc1_mask]
        # arr_y_tpc1 = PromptHits['y'][tpc1_mask]

        tpc2_mask = (PromptHits['x']>0)
        # arr_z_tpc2 = PromptHits['z'][tpc2_mask]
        # arr_y_tpc2 = PromptHits['y'][tpc2_mask]   
        
        h_tpc1_count, zedges_tpc1, yedges_tpc1 = np.histogram2d(PromptHits['z'][tpc1_mask], PromptHits['y'][tpc1_mask], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
        h_tpc2_count, zedges_tpc2, yedges_tpc2 = np.histogram2d(PromptHits['z'][tpc2_mask], PromptHits['y'][tpc2_mask], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
        h_z_count, xedges_zproj, yedges_zproj = np.histogram2d(PromptHits['x'].flatten(), PromptHits['y'].flatten(), bins=[140,280], range=[[x_boundaries[0],x_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
        h_y_count, xedges_yproj, zedges_yproj = np.histogram2d(PromptHits['x'].flatten(), PromptHits['z'].flatten(), bins=[140,140], range=[[z_boundaries[0],z_boundaries[1]],[z_boundaries[0],z_boundaries[1]]])
        
        if i_load == 0:
            total_h_tpc1_count = h_tpc1_count
            total_h_tpc2_count = h_tpc2_count
            total_h_z_count = h_z_count
            total_h_y_count = h_y_count
        else:
            total_h_tpc1_count += h_tpc1_count
            total_h_tpc2_count += h_tpc2_count
            total_h_z_count += h_z_count
            total_h_y_count += h_y_count

    ########
    # TPC1
    ########
    # h_tpc1_count, zedges, yedges = np.histogram2d(PromptHits['z'][tpc1_mask], PromptHits['y'][tpc1_mask], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
    Z, Y = np.meshgrid(zedges_tpc1, yedges_tpc1)
    plt.figure(figsize=(6,10))
    # c_tpc1_count = plt.pcolormesh(Z, Y, h_tpc1_count.T)
    c_tpc1_count = plt.pcolormesh(Z, Y, total_h_tpc1_count.T, vmax=np.sort(total_h_tpc1_count.flatten())[-5])
    plt.colorbar(c_tpc1_count)
    plt.xlabel("z [cm]")
    plt.ylabel("y [cm]")
    plt.title("Occupancy in TPC 1")
    plt.margins(0.2)
    plt.set_cmap("Greens")
    plt.savefig(f"{output_path}TPC1/{output_prefix}TPC1_{f_label}.pdf")
    plt.show()
    
    ########
    # TPC2
    ########
    # h_tpc2_count, zedges, yedges = np.histogram2d(PromptHits['z'][tpc2_mask], PromptHits['y'][tpc2_mask], bins=[140,280], range=[[z_boundaries[0],z_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
    Z, Y = np.meshgrid(zedges_tpc2, yedges_tpc2)
    plt.figure(figsize=(6,10))
    c_tpc2_count = plt.pcolormesh(Z, Y, total_h_tpc2_count.T, vmax=np.sort(total_h_tpc2_count.flatten())[-5])
    plt.colorbar(c_tpc2_count)
    plt.xlabel("z [cm]")
    plt.ylabel("y [cm]")
    plt.title("Occupancy in TPC 2")
    plt.margins(0.2)
    plt.set_cmap("Greens")
    plt.savefig(f"{output_path}TPC2/{output_prefix}TPC2_{f_label}.pdf")
    plt.show()
    
    # ########
    # # z
    # ########
    # h_z_count, zedges, yedges = np.histogram2d(PromptHits['x'].flatten(), PromptHits['y'].flatten(), bins=[140,280], range=[[x_boundaries[0],x_boundaries[1]],[y_boundaries[0],y_boundaries[1]]])
    Z, Y = np.meshgrid(xedges_zproj, yedges_zproj)
    plt.figure(figsize=(6,10))
    c_z_count = plt.pcolormesh(Z, Y, total_h_z_count.T, vmax=np.sort(total_h_z_count.flatten())[-5])
    plt.colorbar(c_z_count)
    plt.xlabel("x [cm]")
    plt.ylabel("y [cm]")
    plt.title("Occupancy (z projection)")
    plt.margins(0.2)
    plt.set_cmap("Greens")
    plt.savefig(f"{output_path}z_proj/{output_prefix}z_proj_{f_label}.pdf")
    plt.show()
    
    ########
    # y
    ########
    # h_y_count, zedges, yedges = np.histogram2d(PromptHits['x'].flatten(), PromptHits['z'].flatten(), bins=[140,140], range=[[z_boundaries[0],z_boundaries[1]],[z_boundaries[0],z_boundaries[1]]])
    Z, Y = np.meshgrid(xedges_yproj, zedges_yproj)
    plt.figure(figsize=(5,4))
    c_y_count = plt.pcolormesh(Z, Y, total_h_y_count.T, vmax=np.sort(total_h_y_count.flatten())[-5])
    plt.colorbar(c_y_count)
    plt.xlabel("x [cm]")
    plt.ylabel("z [cm]")
    plt.title("Occupancy (y projection)")
    plt.margins(0.2)
    plt.set_cmap("Greens")
    plt.savefig(f"{output_path}y_proj/{output_prefix}y_proj_{f_label}.pdf")
    plt.show()